*Load Required Libraries* 

In [1]:
# Libraries for data processing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from pprint import pprint
import tqdm
import datetime

# NLP set-up 
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmaObj = WordNetLemmatizer()

# Libraries for Word Cloud
from wordcloud import WordCloud
from tqdm import tqdm
from collections import Counter

# Libraries for data processing and extraction
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline

# Libraries for Topic Modelling 
import gensim
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\tanay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Users\tanay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices im

In [2]:
df = pd.read_csv('/Users/tanay/Downloads/finalprocessed_data.csv')

In [3]:
df

,Name,Comment,Date,Model,sentiment_score,Source,cleaned_review,sentiment_label
0,Gumpclipper,"Good morning all,I am looking at buying my fir...",2021-10-10,Porsche Macan,5,Pistonheads,good morning looking buying advice essential ...,positive
1,Voodoo Blue,I’m not going to offer an opinion on spec as t...,2021-10-10,Porsche Macan,4,Pistonheads,going offer opinion spec pretty subjective sug...,positive
2,Gumpclipper,Fantastic info. Thank you! Will take a look at...,2021-10-10,Porsche Macan,5,Pistonheads,fantastic info look forum cheer,positive
3,TheBMWDriver,Right now 1% is best you can get on the macan....,2021-10-21,Porsche Macan,4,Pistonheads,right percent best speci find k option make ...,positive
4,neilf,My tip would be to get your name on the deposi...,2021-10-22,Porsche Macan,1,Pistonheads,tip deposit list opc asap way queue decision d...,negative
...,...,...,...,...,...,...,...,...
48940,Binkyjean,The width of the Levante and Grecale are almos...,2022-03-25,Maserati Grecale,2,Forums,width levante inch mirror,negative
48941,Mii,"1,948 mm (76.7 in) is listed on Wiki, should b...",2022-04-02,Maserati Grecale,3,Forums,mm listed wiki width mirror folded http en...,neutral
48942,Harvey Wiggins,"Finally, I found a spec that listed body width...",2022-04-02,Maserati Grecale,3,Forums,finally found spec listed body width mirror wi...,neutral
48943,Mii,"Need to see the gear button control in person,...",2022-04-02,Maserati Grecale,3,Forums,gear button control person non gear lever pref...,neutral


In [4]:
# Filter rows with Model "Porsche Macan"
stelvio_data = df[df['Model'] == 'Alfa Romeo Stelvio']

stelvio_data

,Name,Comment,Date,Model,sentiment_score,Source,cleaned_review,sentiment_label
17104,TurboHatchback,nickfrog said:wemorgan said:TurboHatchback sai...,2017-02-28,Alfa Romeo Stelvio,2,Pistonheads,nickfrog wemorgan turbohatchback woop ugly jac...,negative
17105,underphil,"TurboHatchback said:Woop, yet another ugly jac...",2017-02-28,Alfa Romeo Stelvio,1,Pistonheads,turbohatchback woop ugly jacked uncomfortable ...,negative
17106,dickyf,i stumbled across a launch of this in Courmeye...,2017-02-28,Alfa Romeo Stelvio,5,Pistonheads,stumbled launch courmeyeur week looked good fl...,positive
17107,nickfrog,Kitchski said:Some people who care too much ho...,2017-02-28,Alfa Romeo Stelvio,3,Pistonheads,kitchski e perceive insist trying cram tool su...,neutral
17108,Kitchski,nickfrog said:Kitchski said:Some people who ca...,2017-02-28,Alfa Romeo Stelvio,3,Pistonheads,nickfrog kitchski e perceive insist trying cra...,neutral
...,...,...,...,...,...,...,...,...
25721,cue2,"Lounge is a Fiat trim level, not Alfa!",2016-11-17,Alfa Romeo Stelvio,3,Forums,lounge fiat trim level,neutral
25722,Razor1955,Don't brush against the doors - the paint isn'...,2016-11-17,Alfa Romeo Stelvio,1,Forums,brush door paint dry,negative
25723,Papa Gallo,lets see how many we can fit in here5 so far:w...,2016-11-18,Alfa Romeo Stelvio,5,Forums,let fit here far wavey wavey wavey wavey wavey,positive
25724,Alfamack,"Not too pleased about the décor, can someone d...",2016-11-18,Alfa Romeo Stelvio,2,Forums,pleased décor abut,negative


*LDA Model*

In [5]:
# Create Dictionary
dicton = corpora.Dictionary(stelvio_data['cleaned_review'].str.split())

# Create Corpus
texts = stelvio_data['cleaned_review'].str.split()

# Term Document Frequency
corpus = [dicton.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 2), (29, 1)]


In [6]:
# Build a LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dicton,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [7]:
# Print the keywords in topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"battery" + 0.021*"dealer" + 0.015*"service" + 0.014*"update" + '
  '0.013*"problem" + 0.012*"issue" + 0.011*"start" + 0.010*"work" + '
  '0.010*"thanks" + 0.008*"got"'),
 (1,
  '0.021*"k" + 0.015*"like" + 0.014*"price" + 0.012*"year" + 0.011*"look" + '
  '0.009*"good" + 0.007*"th" + 0.007*"giulia" + 0.007*"review" + '
  '0.007*"month"'),
 (2,
  '0.011*"roof" + 0.009*"brake" + 0.008*"boot" + 0.008*"rear" + 0.008*"bar" + '
  '0.007*"black" + 0.007*"fit" + 0.007*"part" + 0.006*"like" + 0.006*"look"'),
 (3,
  '0.018*"giulia" + 0.012*"model" + 0.010*"veloce" + 0.009*"version" + '
  '0.008*"milano" + 0.008*"edition" + 0.008*"looking" + 0.008*"wheel" + '
  '0.007*"auto" + 0.007*"play"'),
 (4,
  '0.018*"like" + 0.017*"look" + 0.017*"tyre" + 0.009*"mode" + 0.009*"sound" + '
  '0.008*"good" + 0.008*"week" + 0.007*"day" + 0.007*"feel" + 0.006*"ok"'),
 (5,
  '0.018*"com" + 0.017*"www" + 0.016*"forum" + 0.012*"giulia" + 0.010*"system" '
  '+ 0.009*"http" + 0.009*"like" + 0.009*"uk" +

In [8]:
# Calculate the overall Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dicton, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4954512347246145


In [9]:
# Supporting Model 
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dicton, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [10]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 12
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dicton, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results)
    model_results
    pbar.close()

100%|██████████| 600/600 [2:58:31<00:00, 17.85s/it]  


In [11]:
stelvio_results= pd.DataFrame(model_results)
stelvio_results

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.618048
1,75% Corpus,2,0.01,0.31,0.599257
2,75% Corpus,2,0.01,0.61,0.600999
3,75% Corpus,2,0.01,0.91,0.557550
4,75% Corpus,2,0.01,symmetric,0.589304
...,...,...,...,...,...
595,100% Corpus,11,asymmetric,0.01,0.439150
596,100% Corpus,11,asymmetric,0.31,0.479303
597,100% Corpus,11,asymmetric,0.61,0.538180
598,100% Corpus,11,asymmetric,0.91,0.433358


In [12]:
stelvio_results.to_csv('/Users/tanay/Downloads/stelvio_cohenrencef.csv', index=False, encoding="utf-8-sig")

In [13]:
# Number of Topics 3 has the highest coherence score (0.66), therefore we continue with that
num_topics = 3

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dicton,
                                           num_topics=num_topics, 
                                           random_state=42,
                                           chunksize=100,
                                           passes=15,
                                           alpha=0.02,
                                           eta=0.9)

In [14]:
# Print the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"dealer" + 0.006*"like" + 0.006*"giulia" + 0.005*"battery" + '
  '0.005*"good" + 0.005*"service" + 0.005*"got" + 0.004*"engine" + '
  '0.004*"issue" + 0.004*"year"'),
 (1,
  '0.011*"look" + 0.008*"like" + 0.008*"giulia" + 0.006*"com" + 0.006*"www" + '
  '0.005*"uk" + 0.005*"black" + 0.004*"good" + 0.004*"red" + 0.004*"wheel"'),
 (2,
  '0.009*"tyre" + 0.006*"wheel" + 0.005*"like" + 0.004*"rear" + 0.004*"door" + '
  '0.003*"boot" + 0.003*"light" + 0.003*"look" + 0.003*"seat" + '
  '0.003*"winter"')]


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, dicton)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.097277 -0.023601       1        1  56.582533
1      0.069751 -0.069757       2        1  24.399613
2      0.027526  0.093359       3        1  19.017854, topic_info=         Term        Freq       Total Category  logprob  loglift
2432     tyre  265.000000  265.000000  Default  30.0000  30.0000
165      look  730.000000  730.000000  Default  29.0000  29.0000
1184      com  235.000000  235.000000  Default  28.0000  28.0000
1197      www  235.000000  235.000000  Default  27.0000  27.0000
1001   dealer  613.000000  613.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
580      work   62.145108  369.156601   Topic3  -6.1409  -0.1220
536       way   62.447534  380.325795   Topic3  -6.1361  -0.1469
342   problem   57.437918  364.572451   Topic3  -6.2197  -0.1882
60       good   62.441438  624.953036   Topic3  -6.1362  -0.6437
275     right   52.130386  220.303384   Topic3  -6.3166   0.2185

[218 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4961      1  0.966757        agm
4961      2  0.026129        agm
4961      3  0.026129        agm
1609      1  0.052175  alfaromeo
1609      2  0.939157  alfaromeo
...     ...       ...        ...
5337      2  0.043130      xenon
5337      3  0.905722      xenon
144       1  0.909726       year
144       2  0.085021       year
144       3  0.005668       year

[522 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])